Creating WDIdb

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import pprint

In [2]:
df = pd.read_excel('/home/lolo/WDI_excel/WDIEXCEL.xlsx', sheet_name='Data')

In [3]:
INDS = np.append(df['Indicator Name'][464+0:464+20], df['Indicator Name'][492+0:492+16])
INDs = ''
for i in range(len(INDS)):
    if i > 0:
       INDs = INDs + ';' + INDS[i]
    else:
        INDs = INDs + INDS[i]
print(INDs)
print(len(INDS), len(INDs.split(';')))

GDP (constant 2010 US$);GDP (constant LCU);GDP (current LCU);GDP (current US$);GDP deflator (base year varies by country);GDP deflator: linked series (base year varies by country);GDP growth (annual %);GDP per capita (constant 2010 US$);GDP per capita (constant LCU);GDP per capita (current LCU);GDP per capita (current US$);GDP per capita growth (annual %);GDP per capita, PPP (constant 2017 international $);GDP per capita, PPP (current international $);GDP per person employed (constant 2017 PPP $);GDP per unit of energy use (constant 2017 PPP $ per kg of oil equivalent);GDP per unit of energy use (PPP $ per kg of oil equivalent);GDP, PPP (constant 2017 international $);GDP, PPP (current international $);GDP: linked series (current LCU);GNI (constant 2010 US$);GNI (constant LCU);GNI (current LCU);GNI (current US$);GNI growth (annual %);GNI per capita (constant 2010 US$);GNI per capita (constant LCU);GNI per capita (current LCU);GNI per capita growth (annual %);GNI per capita, Atlas metho

In [4]:
CNS = df['Country Name']
CNs = [] 
for i in CNS: 
    if i.lower() not in CNs: 
        CNs.append(i.lower())

In [5]:
yrs = ['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']
YRs = ''
for i in range(len(yrs)):
    if i > 0:
       YRs = YRs + ';' + yrs[i]
    else:
        YRs = YRs + yrs[i]
print(YRs)   

1960;1961;1962;1963;1964;1965;1966;1967;1968;1969;1970;1971;1972;1973;1974;1975;1976;1977;1978;1979;1980;1981;1982;1983;1984;1985;1986;1987;1988;1989;1990;1991;1992;1993;1994;1995;1996;1997;1998;1999;2000;2001;2002;2003;2004;2005;2006;2007;2008;2009;2010;2011;2012;2013;2014;2015;2016;2017;2018;2019


In [6]:
l = 0
rg = CNs[l:]
l1 = 20
l2 = 16
DATA = np.zeros((len(rg), l1+l2, len(yrs))) 
for i in range(l, len(CNs)):
    for all in range(l1):
        jj = 0
        for j in yrs:
            if ~np.isnan(df[j][1431*i+464+all]):
                DATA[i-l,all,jj] = df[j][1431*i+464+all]
            jj += 1    
    for all in range(l2):
        jj = 0
        for j in yrs:  
            if ~np.isnan(df[j][1431*i+492+all]):
                DATA[i-l,all+l1,jj] = df[j][1431*i+492+all]
            jj += 1    

In [7]:
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

sqlite3.register_adapter(np.ndarray, adapt_array)
sqlite3.register_converter('array', convert_array)

In [8]:
conn = sqlite3.connect('WDIdb.sqlite', detect_types=sqlite3.PARSE_DECLTYPES)
cur = conn.cursor() 

In [9]:
sql_command ="""
DROP TABLE IF EXISTS measurements;
CREATE TABLE WDI (
    id INTEGER PRIMARY KEY,
    name VARCHAR,
    years VARCHAR,    
    indicators VARCHAR,
    arr array);
"""
cur.executescript(sql_command)
conn.commit()

In [10]:
for i in range(len(rg)):
    cur.execute('INSERT INTO WDI (name, years, indicators, arr) values ("' + CNs[i] + '", "' + YRs + '", "' + INDs + '", ?)', (DATA[i],))
    conn.commit()

In [11]:
cur.execute('SELECT * FROM WDI WHERE name="mexico"')
data = cur.fetchone()
pprint.pprint(data)
#plt.imshow(data[4])

(175,
 'mexico',
 '1960;1961;1962;1963;1964;1965;1966;1967;1968;1969;1970;1971;1972;1973;1974;1975;1976;1977;1978;1979;1980;1981;1982;1983;1984;1985;1986;1987;1988;1989;1990;1991;1992;1993;1994;1995;1996;1997;1998;1999;2000;2001;2002;2003;2004;2005;2006;2007;2008;2009;2010;2011;2012;2013;2014;2015;2016;2017;2018;2019',
 'GDP (constant 2010 US$);GDP (constant LCU);GDP (current LCU);GDP (current '
 'US$);GDP deflator (base year varies by country);GDP deflator: linked series '
 '(base year varies by country);GDP growth (annual %);GDP per capita (constant '
 '2010 US$);GDP per capita (constant LCU);GDP per capita (current LCU);GDP per '
 'capita (current US$);GDP per capita growth (annual %);GDP per capita, PPP '
 '(constant 2017 international $);GDP per capita, PPP (current international '
 '$);GDP per person employed (constant 2017 PPP $);GDP per unit of energy use '
 '(constant 2017 PPP $ per kg of oil equivalent);GDP per unit of energy use '
 '(PPP $ per kg of oil equivalent);GDP, PPP 